In [1]:
#Imports
import sys
sys.path.append('../scripts/')

import os
import numpy as np
import pandas as pd
from bayes_optimizer import BayesianOptimizer
from helpers import FE, FM, FD, find_global_max
from utils import load_data, preprocess
from make_graphs import plot_suggestions, plot_convergence, plot_dataset

import warnings
warnings.filterwarnings("ignore")


In [2]:
# 2) Configuration: choose function, feature columns, target, data path
FUNC_NAME = "FE"                                # or "FM", "FD"
FUNC = {"FE": FE, "FM": FM, "FD": FD}[FUNC_NAME]
FEATURE_COLUMNS = [f"x{i+1}" for i in range(2)]  # e.g. 5-D
TARGET_COLUMN = "y"
DATA_PATH = f"../datasets/synth_data/{FUNC_NAME}_d{len(FEATURE_COLUMNS)}_N50.json"


In [3]:
# 3) Load & preprocess
x_raw, y, df = load_data(DATA_PATH, FEATURE_COLUMNS, TARGET_COLUMN)
x_init, scaler = preprocess(x_raw)
bounds = [[0,10]] * len(FEATURE_COLUMNS)


In [4]:
# 4) Instantiate and run the optimizer
bo = BayesianOptimizer(
    func=FUNC,
    feature_columns=FEATURE_COLUMNS,
    target_column=TARGET_COLUMN,
    surrogate_kind="gp",
    opt_method="grad_opt",
    acq="ei",
    seed=37
)
x_all, y_all, pred_y, y_train_max = bo.optimize(x_init, y, bounds, n_iter=30)
bounds_arr = np.array(bounds)
global_x, global_max = find_global_max(FUNC, bounds_arr)


In [5]:
# 5) Convergence plot
plot_convergence(pred_y, y_train_max, global_max, f"{FUNC_NAME}_d{len(FEATURE_COLUMNS)}", out_dir="../results/synth_results/")


In [6]:
# 6) Full-dataset scatter
#plot_dataset(df, FEATURE_COLUMNS[0], TARGET_COLUMN, FUNC_NAME, len(FEATURE_COLUMNS), out_dir="../results/synth_results/")


In [7]:
# 7) Suggested‐points scatter
#   best points across iterations:
best_idx = int(np.argmax(pred_y))
best_x = x_all[-len(pred_y):][best_idx]
best_y = pred_y[best_idx]
plot_suggestions(
    df,
    FEATURE_COLUMNS[0],
    global_max,
    [best_x[0]],
    [best_y],
    TARGET_COLUMN,
    FUNC_NAME,
    len(FEATURE_COLUMNS),
    out_dir="../results/synth_results/"
)